In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

from exp.nb_03 import *

In [3]:
x_train,y_train,x_valid,y_valid = get_data()
train_ds,valid_ds = Dataset(x_train, y_train),Dataset(x_valid, y_valid)
nh,bs = 50,64
c = y_train.max().item()+1
loss_func = F.cross_entropy

In [4]:
class DataBunch():
    def __init__(self, train_dl, valid_dl, c=None):
        self.train_dl=train_dl
        self.valid_dl=valid_dl
        self.c = c
    @property
    def train_ds(self): return self.train_dl.dataset
    
    @property
    def valid_ds(self): return self.valid_dl.dataset


In [5]:
data = DataBunch(*get_dls(train_ds,valid_ds,bs),c)

In [6]:
train_ds.x.shape

torch.Size([50000, 784])

In [7]:
def get_model(data,lr=0.5,nh=50):
    m=data.train_ds.x.shape[1]
    model= nn.Sequential(nn.Linear(m,nh),nn.ReLU(), nn.Linear(nh,data.c))
    return model, optim.SGD(model.parameters(),lr=lr)
class Learner():
    def __init__(self, model, opt,loss_func,data):
        self.model=model
        self.opt =opt
        self.loss_func=loss_func
        self.data=data

In [8]:
learn = Learner(*get_model(data),loss_func,data)

In [9]:
def fit(epochs,learn):
    for epoch in range(epochs):
        learn.model.train()
        for xb,yb in learn.data.train_dl:
            loss= learn.loss_func(learn.model(xb),yb)
            loss.backward()
            learn.opt.step()
            learn.opt.zero_grad()
        learn.model.eval()
        with torch.no_grad():
            tot_loss, tot_acc = 0.,0.
            for xb, yb in learn.data.valid_dl:
                pred = learn.model(xb)
                tot_loss += learn.loss_func(pred,yb)
                tot_acc += accuracy(pred,yb)
        nv = len(learn.data.valid_dl)
        print(epoch,tot_loss/nv,tot_acc/nv)
        return tot_loss/nv, tot_acc/nv

In [10]:
loss,acc=fit(1,learn)

0 tensor(0.2432) tensor(0.9252)


In [11]:
def one_batch(xb,yb):
    pred = model(xb)
    loss = loss_func(pred,yb)
    loss.backward()
    learn.opt.step()
    learn.opt.zero_grad()
def fit():
    for epoch in range(epochs):
        for b in train_dl: one_batch()
    

In [29]:
def one_batch(xb,yb,cb):
    if not cb.begin_batch(xb,yb): return
    loss = cb.learn.loss_func(cb.learn.model(xb),yb)
    if not cb.after_loss(loss): return
    loss.backward()
    if cb.after_backward(): cb.learn.opt.step()
    if cb.after_step(): cb.learn.opt.zero_grad()

def all_batches(dl,cb):
    for xb,yb in dl:
        one_batch(xb,yb,cb)
        if cb.do_stop(): return

In [30]:
def fit(epochs,learn,cb):
    if not cb.begin_fit(learn): return
    for epoch in range(epochs):
        if not cb.begin_epoch(epoch): continue
        all_batches(learn.data.train_dl,cb)
        
        if  cb.begin_validate():
            with torch.no_grad():all_batches(learn.data.valid_dl, cb)
        if cb.do_stop() or not cb.after_epoch(): break
    cb.after_fit()

In [31]:
class Callback():
    def begin_fit(self, learn):
        self.learn = learn
        return True
    def after_fit(self): return True
    def begin_epoch(self, epoch):
        self.epoch=epoch
        return True
    def begin_validate(self): return True
    def after_epoch(self): return True
    def begin_batch(self, xb, yb):
        self.xb,self.yb = xb,yb
        return True
    def after_loss(self, loss):
        self.loss = loss
        return True
    def after_backward(self): return True
    def after_step(self): return True

In [56]:
class CallbackHandler():
    def __init__(self,cbs=None):
        self.cbs = cbs if cbs else []

    def begin_fit(self, learn):
        self.learn,self.in_train = learn,True
        learn.stop = False
        res = True
        ## added by me
        num=0
        for cb in self.cbs: 
            res = res and cb.begin_fit(learn)
            ## added by me
            num += 1
        ## added by me
        print(cb,num)
        return res

    def after_fit(self):
        res = not self.in_train
        for cb in self.cbs: res = res and cb.after_fit()
        return res
    
    def begin_epoch(self, epoch):
        self.learn.model.train()
        self.in_train=True
        res = True
        for cb in self.cbs: res = res and cb.begin_epoch(epoch)
        return res

    def begin_validate(self):
        self.learn.model.eval()
        self.in_train=False
        res = True
        for cb in self.cbs: res = res and cb.begin_validate()
        return res

    def after_epoch(self):
        res = True
        for cb in self.cbs: res = res and cb.after_epoch()
        return res
    
    def begin_batch(self, xb, yb):
        res = True
        for cb in self.cbs: res = res and cb.begin_batch(xb, yb)
        return res

    def after_loss(self, loss):
        res = self.in_train
        for cb in self.cbs: res = res and cb.after_loss(loss)
        return res

    def after_backward(self):
        res = True
        for cb in self.cbs: res = res and cb.after_backward()
        return res

    def after_step(self):
        res = True
        for cb in self.cbs: res = res and cb.after_step()
        return res
    def do_stop(self):
        try:
            return self.learn.stop
        finally: self.learn.stop = False
        

In [57]:
class TestCallback(Callback):
    def begin_fit(self,learn):
        super().begin_fit(learn)
        self.n_iters = 0
        return True
    def after_step(self):
        self.n_iters += 1
        print(self.n_iters)
        if self.n_iters >= 10 : self.learn.stop = True
        return True

In [58]:
fit(1, learn, cb=CallbackHandler([TestCallback()]))

<__main__.TestCallback object at 0x7fe1f71cd460> 1
1
2
3
4
5
6
7
8
9
10
